In [21]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from math import radians, cos, sin, asin, sqrt
from sklearn.preprocessing import MinMaxScaler

def year_month_extract(df, col):
    df[col + "_year"] = df[col].dt.year
    df[col + "_month"] = df[col].dt.month
    df[col + "_day"] = df[col].dt.day

    return year_month_extract

def duration_minutes(df): 
    df["Duration"] = df["Duration"].str.replace("h", "*60").str.replace(' ','+').str.replace("m","*1").apply(eval)

    return df 

def dep_arrival_extract(df, col):
    df[col + "_hour"] = df[col].dt.hour
    df[col + "_minute"] = df[col].dt.minute
    return dep_arrival_extract

def geo_extraction(df, col1 = "Source", col2 = "Destination"):

    """"
    This function get the coordinates for the airports based on the city efficiently
    (using minimal calls from geopy API based only on unique cities instead of iterating 
    over all the cities ) 
    """

    src = list(df[col1].unique())
    des = list(df[col2].unique())

    cities = []

    for i in src, des:
      cities.append(i)
    cities = list(set(np.ravel(cities)))

    airport_name = []

    for city in cities:
        airport_name.append(city + " International Airport")

    airports_dict = dict(zip(cities, airport_name))

    airport_src = []
    airport_des = []

    for i, j in zip(df[col1], df[col2]):
        for k, v in airports_dict.items():
            if i == k:
                airport_src.append(v)
            if j == k:
                airport_des.append(v)

    df[col1 + "_Airport"] = airport_src
    df[col2 + "_Airport"] = airport_des

    geolocator = Nominatim(user_agent="my-flight-fare")
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

    lat = []
    lon = []

    for airport in airport_name:
        location = geolocator.geocode(airport)
        lat.append(location.latitude)
        lon.append(location.longitude)

    geo = tuple(zip(lat, lon))

    airport_geo = dict(zip(airport_name, geo))

    src_geo = []
    des_geo = []

    for src_airport, des_airport in zip(df[col1 + "_Airport"], df[col2 + "_Airport"]):
        for k, v in airport_geo.items():
            if src_airport == k:
                src_geo.append(v)
            if des_airport == k:
                des_geo.append(v)

    df[[col1 + '_Lat', col1 + '_Lon']] = pd.DataFrame(src_geo, index=df.index)
    df[[col2 + '_Lat', col2 + '_Lon']] = pd.DataFrame(des_geo, index=df.index)

    return df 

def haversine_dist(lon1, lat1, lon2, lat2):

    lat1 = np.radians(lat1)
    lat2 = np.radians(lat2)
    lon1 = np.radians(lon1)
    lon2 = np.radians(lon2)

    # Haversine formula 
    dlat = lat2 - lat1
    dlon = lon2 - lon1 
    a = np.sin(dlat / 2.0)** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0) ** 2

    c = 2 * np.arcsin(np.sqrt(a))

    r = 6371 # radius of earth in km

    distance = c * r

    return distance

def add_distance(df):
    df["flight_distance"] = haversine_dist(df["Source_Lon"], 
    df["Source_Lat"], 
    df["Destination_Lon"], df["Destination_Lat"])

    df.drop(["Source_Lat", "Source_Lon", "Destination_Lat", "Destination_Lon"], axis = 1, inplace=True)
    return df

def x_var(df):
    df = df.drop(["Price"], axis=1, errors="ignore")

    return df

def y_var(df):
    df = df["Price"]

    return df


def preprocess_data(df):

    df = pd.read_excel(df)

    df["Date_of_Journey"] = pd.to_datetime(df["Date_of_Journey"])

    year_month_extract(df, "Date_of_Journey")

    df = df.dropna()
    df = df.reset_index(drop = True)
    df.drop(["Date_of_Journey", "Route", "Additional_Info"], axis = 1, inplace=True)

    duration_minutes(df)

    df["Dep_Time"] = pd.to_datetime(df["Dep_Time"])
    df["Arrival_Time"] = pd.to_datetime(df["Arrival_Time"])

    df.replace({"non-stop": 0, "1 stop": 1, "2 stops": 2, "3 stops": 3, "4 stops": 4}, 
    inplace = True)

    dep_arrival_extract(df, "Dep_Time")
    dep_arrival_extract(df, "Arrival_Time")

    df.drop(["Dep_Time", "Arrival_Time"], axis = 1, inplace = True)

    df.rename(columns = {'Total_Stops':'stop','Date_of_Journey_month':'month', 
                     'Date_of_Journey_day':'day', "Dep_Time_hour": "dep_hr", 
                     "Dep_Time_minute":"dep_min", "Arrival_Time_hour":"arr_hr", 
                     "Arrival_Time_minute":"arr_min"}, inplace = True)

    df = df.replace("Delhi", "New Delhi")
    df = df.replace("Banglore", "Bengaluru")

    geo_extraction(df, "Source", "Destination")

    df.drop(["Source_Airport", "Destination_Airport"], axis = 1, inplace = True)

    add_distance(df)

    dummy_airline = pd.get_dummies(df["Airline"], prefix="Airline", drop_first= True)
    dummy_source = pd.get_dummies(df["Source"], prefix="Source", drop_first= True)
    dummy_destination = pd.get_dummies(df["Destination"], prefix="Destination")

    df = pd.concat([df, dummy_airline, dummy_source, dummy_destination], axis=1)

    df.drop(["Airline", "Source", "Destination", "Date_of_Journey_year", "Airline_Vistara Premium economy", "Airline_Trujet"], axis = 1, inplace = True, errors='ignore')

    #df.to_csv("test_cleaned.csv", index=False)
    return df


In [3]:
preprocess_data("D:/Internship/flight_fare/flight-fare-prediction/data/Test_set.xlsx")

In [22]:
df_train = preprocess_data("D:/Internship/flight_fare/flight-fare-prediction/data/Data_Train.xlsx")
df_train

,Duration,stop,Price,month,day,dep_hr,dep_min,arr_hr,arr_min,flight_distance,...,Airline_Vistara,Source_Chennai,Source_Kolkata,Source_Mumbai,Source_New Delhi,Destination_Bengaluru,Destination_Cochin,Destination_Hyderabad,Destination_Kolkata,Destination_New Delhi
0,170,0,3897,3,24,22,20,1,10,1708.999039,...,0,0,0,0,0,0,0,0,0,1
1,445,2,7662,1,5,5,50,13,15,1546.957213,...,0,0,1,0,0,1,0,0,0,0
2,1140,2,13882,9,6,9,25,4,25,2047.335727,...,0,0,0,0,1,0,1,0,0,0
3,325,1,6218,12,5,18,5,23,30,1546.957213,...,0,0,1,0,0,1,0,0,0,0
4,285,1,13302,1,3,16,50,21,35,1708.999039,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10677,150,0,4107,9,4,19,55,22,25,1546.957213,...,0,0,1,0,0,1,0,0,0,0
10678,155,0,4145,4,27,20,45,23,20,1546.957213,...,0,0,1,0,0,1,0,0,0,0
10679,180,0,7229,4,27,8,20,11,20,1708.999039,...,0,0,0,0,0,0,0,0,0,1
10680,160,0,12648,1,3,11,30,14,10,1708.999039,...,1,0,0,0,0,0,0,0,0,1


In [23]:
df_test = preprocess_data("D:/Internship/flight_fare/flight-fare-prediction/data/Test_set.xlsx")
df_test

,Duration,stop,month,day,dep_hr,dep_min,arr_hr,arr_min,flight_distance,Airline_Air India,...,Airline_Vistara,Source_Chennai,Source_Kolkata,Source_Mumbai,Source_New Delhi,Destination_Bengaluru,Destination_Cochin,Destination_Hyderabad,Destination_Kolkata,Destination_New Delhi
0,655,1,6,6,17,30,4,25,2047.335727,0,...,0,0,0,0,1,0,1,0,0,0
1,240,1,12,5,6,20,10,20,1546.957213,0,...,0,0,1,0,0,1,0,0,0,0
2,1425,1,5,21,19,15,19,0,2047.335727,0,...,0,0,0,0,1,0,1,0,0,0
3,780,1,5,21,8,0,21,0,2047.335727,0,...,0,0,0,0,1,0,1,0,0,0
4,170,0,6,24,23,55,2,45,1708.999039,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2666,1435,1,6,6,20,30,20,25,1546.957213,1,...,0,0,1,0,0,1,0,0,0,0
2667,155,0,3,27,14,20,16,55,1546.957213,0,...,0,0,1,0,0,1,0,0,0,0
2668,395,1,6,3,21,50,4,25,2047.335727,0,...,0,0,0,0,1,0,1,0,0,0
2669,915,1,6,3,4,0,19,15,2047.335727,1,...,0,0,0,0,1,0,1,0,0,0


In [24]:
x_train = x_var(df_train)
x_test = x_var(df_test)

In [25]:
y_train = y_var(df_train)

In [30]:
x_test

,Duration,stop,month,day,dep_hr,dep_min,arr_hr,arr_min,flight_distance,Airline_Air India,...,Airline_Vistara,Source_Chennai,Source_Kolkata,Source_Mumbai,Source_New Delhi,Destination_Bengaluru,Destination_Cochin,Destination_Hyderabad,Destination_Kolkata,Destination_New Delhi
0,655,1,6,6,17,30,4,25,2047.335727,0,...,0,0,0,0,1,0,1,0,0,0
1,240,1,12,5,6,20,10,20,1546.957213,0,...,0,0,1,0,0,1,0,0,0,0
2,1425,1,5,21,19,15,19,0,2047.335727,0,...,0,0,0,0,1,0,1,0,0,0
3,780,1,5,21,8,0,21,0,2047.335727,0,...,0,0,0,0,1,0,1,0,0,0
4,170,0,6,24,23,55,2,45,1708.999039,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2666,1435,1,6,6,20,30,20,25,1546.957213,1,...,0,0,1,0,0,1,0,0,0,0
2667,155,0,3,27,14,20,16,55,1546.957213,0,...,0,0,1,0,0,1,0,0,0,0
2668,395,1,6,3,21,50,4,25,2047.335727,0,...,0,0,0,0,1,0,1,0,0,0
2669,915,1,6,3,4,0,19,15,2047.335727,1,...,0,0,0,0,1,0,1,0,0,0


In [31]:
from sklearn.preprocessing import MinMaxScaler

col_to_scale = ['Duration', 'flight_distance']

scaler = MinMaxScaler()
x_train[col_to_scale] = scaler.fit_transform(x_train[col_to_scale])

In [32]:
x_test[col_to_scale] = scaler.transform(x_test[col_to_scale])

In [34]:
x_test

,Duration,stop,month,day,dep_hr,dep_min,arr_hr,arr_min,flight_distance,Airline_Air India,...,Airline_Vistara,Source_Chennai,Source_Kolkata,Source_Mumbai,Source_New Delhi,Destination_Bengaluru,Destination_Cochin,Destination_Hyderabad,Destination_Kolkata,Destination_New Delhi
0,0.227671,1,6,6,17,30,4,25,1.000000,0,...,0,0,0,0,1,0,1,0,0,0
1,0.082312,1,12,5,6,20,10,20,0.648569,0,...,0,0,1,0,0,1,0,0,0,0
2,0.497373,1,5,21,19,15,19,0,1.000000,0,...,0,0,0,0,1,0,1,0,0,0
3,0.271454,1,5,21,8,0,21,0,1.000000,0,...,0,0,0,0,1,0,1,0,0,0
4,0.057793,0,6,24,23,55,2,45,0.762376,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2666,0.500876,1,6,6,20,30,20,25,0.648569,1,...,0,0,1,0,0,1,0,0,0,0
2667,0.052539,0,3,27,14,20,16,55,0.648569,0,...,0,0,1,0,0,1,0,0,0,0
2668,0.136602,1,6,3,21,50,4,25,1.000000,0,...,0,0,0,0,1,0,1,0,0,0
2669,0.318739,1,6,3,4,0,19,15,1.000000,1,...,0,0,0,0,1,0,1,0,0,0


In [35]:
from xgboost import XGBRegressor
reg_xgb = XGBRegressor(colsample_bytree=0.8, learning_rate=0.5)
reg_xgb.fit(x_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.5, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [36]:
y_pred = reg_xgb.predict(x_test)

In [37]:
y_pred

array([11044.297 ,  4317.2295, 14019.403 , ..., 15039.472 , 11427.089 ,
        8019.21  ], dtype=float32)